In [14]:
# importando libs
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import MinMaxScaler
from sklearn import datasets, linear_model
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib.pyplot as plt
import os
from datetime import datetime
from datetime import timedelta
import yfinance as yf
import pickle
from prophet import Prophet
from prophet.serialize import model_to_json, model_from_json
from prophet import plot
import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

In [15]:
DAYS_PREDICT = 15

In [16]:
date_today = datetime.today().strftime("%Y-%m-%d")
data_init = "2014-01-01"

df_sol = yf.download("SOL-USD", data_init, date_today)

df_sol.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2020-04-10,0.832005,1.313487,0.694187,0.951054,0.951054,87364276
2020-04-11,0.951054,1.049073,0.765020,0.776819,0.776819,43862444
2020-04-12,0.785448,0.956670,0.762426,0.882507,0.882507,38736897
2020-04-13,0.890760,0.891603,0.773976,0.777832,0.777832,18211285
2020-04-14,0.777832,0.796472,0.628169,0.661925,0.661925,16747614


In [17]:
df_sol.reset_index(inplace=True)
df_sol.tail()

,Date,Open,High,Low,Close,Adj Close,Volume
885,2022-09-12,34.967442,38.169079,34.507923,37.424183,37.424183,1701399970
886,2022-09-13,37.421608,38.870384,32.870453,33.430817,33.430817,1785181627
887,2022-09-14,33.018276,34.283195,32.870453,34.091572,34.091572,1003604224
888,2022-09-15,34.094292,34.715267,32.721893,33.047684,33.047684,1139377803
889,2022-09-16,33.044594,33.203125,31.641165,32.208858,32.208858,765952525


In [18]:
df = df_sol[["Date", "Adj Close"]]
df.rename(columns= { "Date": "ds", "Adj Close": "y" }, inplace=True)

In [19]:
df.tail()

,ds,y
885,2022-09-12,37.424183
886,2022-09-13,33.430817
887,2022-09-14,34.091572
888,2022-09-15,33.047684
889,2022-09-16,32.208858


In [20]:
assert df.isnull().sum().sum() == 0
assert df['ds'].isnull().sum() == 0
assert df['y'].isnull().sum() == 0

In [21]:
df.dtypes

ds    datetime64[ns]
y            float64
dtype: object

In [22]:
model = Prophet(seasonality_mode="multiplicative")
model.fit(df)

00:34:56 - cmdstanpy - INFO - Chain [1] start processing
00:34:56 - cmdstanpy - INFO - Chain [1] done processing


In [23]:
df_future = model.make_future_dataframe(periods=DAYS_PREDICT)
df_future.tail()

,ds
900,2022-09-27
901,2022-09-28
902,2022-09-29
903,2022-09-30
904,2022-10-01


In [24]:
# predict 
predict = model.predict(df_future)
predict

,ds,trend,yhat_lower,yhat_upper,trend_lower,trend_upper,multiplicative_terms,multiplicative_terms_lower,multiplicative_terms_upper,weekly,weekly_lower,weekly_upper,yearly,yearly_lower,yearly_upper,additive_terms,additive_terms_lower,additive_terms_upper,yhat
0,2020-04-10,1.245891,-9.657348,11.012521,1.245891,1.245891,-0.001118,-0.001118,-0.001118,-0.006988,-0.006988,-0.006988,0.005870,0.005870,0.005870,0.0,0.0,0.0,1.244497
1,2020-04-11,1.246743,-9.771355,11.978694,1.246743,1.246743,0.010806,0.010806,0.010806,0.005577,0.005577,0.005577,0.005229,0.005229,0.005229,0.0,0.0,0.0,1.260215
2,2020-04-12,1.247594,-9.694765,11.484776,1.247594,1.247594,0.009497,0.009497,0.009497,0.005862,0.005862,0.005862,0.003634,0.003634,0.003634,0.0,0.0,0.0,1.259442
3,2020-04-13,1.248446,-9.055260,12.276971,1.248446,1.248446,-0.005150,-0.005150,-0.005150,-0.006307,-0.006307,-0.006307,0.001157,0.001157,0.001157,0.0,0.0,0.0,1.242016
4,2020-04-14,1.249297,-9.821671,11.766461,1.249297,1.249297,-0.008790,-0.008790,-0.008790,-0.006660,-0.006660,-0.006660,-0.002130,-0.002130,-0.002130,0.0,0.0,0.0,1.238316
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
900,2022-09-27,17.305703,14.650236,35.198903,17.233432,17.387433,0.408339,0.408339,0.408339,-0.006660,-0.006660,-0.006660,0.414999,0.414999,0.414999,0.0,0.0,0.0,24.372295
901,2022-09-28,16.745770,12.879595,34.408815,16.645399,16.852163,0.403125,0.403125,0.403125,0.006506,0.006506,0.006506,0.396619,0.396619,0.396619,0.0,0.0,0.0,23.496408
902,2022-09-29,16.185838,11.772146,32.145951,16.035874,16.319890,0.381594,0.381594,0.381594,0.002011,0.002011,0.002011,0.379584,0.379584,0.379584,0.0,0.0,0.0,22.362264
903,2022-09-30,15.625906,10.087960,31.815869,15.424279,15.803352,0.357207,0.357207,0.357207,-0.006988,-0.006988,-0.006988,0.364196,0.364196,0.364196,0.0,0.0,0.0,21.207594


In [25]:
predict[["ds", "yhat_lower", "yhat_upper", "yhat"]].tail()

,ds,yhat_lower,yhat_upper,yhat
900,2022-09-27,14.650236,35.198903,24.372295
901,2022-09-28,12.879595,34.408815,23.496408
902,2022-09-29,11.772146,32.145951,22.362264
903,2022-09-30,10.087960,31.815869,21.207594
904,2022-10-01,10.640943,31.167402,20.434006


In [26]:
# Salvar modelo
with open('model_sol_usd.pckl', 'wb') as fout:
    pickle.dump(model, fout)